In [4]:
# Database insertion 
from sqlalchemy.exc import IntegrityError
from sqlalchemy import create_engine
import pandas as pd 
engine = create_engine('postgres://miro:@localhost/qetch')


In [2]:
rs = engine.execute('SELECT * from measurementdescription;')
[i for i in rs]

[]

In [60]:
columns = ["sname" , "snum","time","value", "metadata" ]
import json

In [87]:
df = pd.read_csv('../plots/qetch/7D-30-[1-10]-[SEX]-lcm.out-raw-sliding_windows.csv')
sname = "RAW-SLIDING_WINDOWS"
df.customers = df.customers.apply(lambda x: dict(customers=x) ).apply(json.dumps)
df.granularity = df.granularity.apply(lambda x: f'{sname}-{x}')
df["sname"] = sname

df.rename(columns = {
    "period_y":"time",
    "interest_gain":"value",
    "customers":"metadata",
    "granularity":"snum"
},inplace=True)
# df.head()

In [88]:
df.to_sql("measurement",con=engine,if_exists="append",index=False)

In [92]:
series_description = df[["snum","sname"]].drop_duplicates().rename(columns = {"snum":'measurementseries',"sname":"measurementkey"})
series_description["description"] = series_description["measurementseries"] 
series_description.to_sql("measurementseriesdescription",con=engine,if_exists="append",index=False)
series_description

,measurementseries,measurementkey,description
0,RAW-SLIDING_WINDOWS-1W,RAW-SLIDING_WINDOWS,RAW-SLIDING_WINDOWS-1W
24,RAW-SLIDING_WINDOWS-2W,RAW-SLIDING_WINDOWS,RAW-SLIDING_WINDOWS-2W
36,RAW-SLIDING_WINDOWS-3W,RAW-SLIDING_WINDOWS,RAW-SLIDING_WINDOWS-3W
44,RAW-SLIDING_WINDOWS-4W,RAW-SLIDING_WINDOWS,RAW-SLIDING_WINDOWS-4W


In [85]:
series_description.to_sql(name="measurementseriesdescription",con=engine,if_exists="append",index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk1"
DETAIL:  Key (measurementkey, measurementseries)=(RAW-SLIDING_WINDOWS-1W, RAW-SLIDING_WINDOWS) already exists.

[SQL: INSERT INTO measurementseriesdescription (measurementkey, measurementseries, description) VALUES (%(measurementkey)s, %(measurementseries)s, %(description)s)]
[parameters: ({'measurementkey': 'RAW-SLIDING_WINDOWS-1W', 'measurementseries': 'RAW-SLIDING_WINDOWS', 'description': 'RAW-SLIDING_WINDOWS'}, {'measurementkey': 'RAW-SLIDING_WINDOWS-2W', 'measurementseries': 'RAW-SLIDING_WINDOWS', 'description': 'RAW-SLIDING_WINDOWS'}, {'measurementkey': 'RAW-SLIDING_WINDOWS-3W', 'measurementseries': 'RAW-SLIDING_WINDOWS', 'description': 'RAW-SLIDING_WINDOWS'}, {'measurementkey': 'RAW-SLIDING_WINDOWS-4W', 'measurementseries': 'RAW-SLIDING_WINDOWS', 'description': 'RAW-SLIDING_WINDOWS'})]
(Background on this error at: http://sqlalche.me/e/gkpj)